In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click to toggle on/off the raw code."></form>''')

# <a> Data analysis report </a>

The following analysis has been conducted on the website traffic generated by a tool similar to Google Analytics. There are 11 fields in the dataset. Each of these fields give us an idea about how the user is interacting with a website, from where they belong and the device with which they are interacting the website. 

### <a id='main'><h3>Table of Contents</h3></a>

- [Importing the essential libraries](#lib)
- [Reading and modifying the data](#re)
- [How much time does someone typically spend on the web page?](#typical)
    - [(a) (i) From which countries does the website experience typical session durations?](#typical_countries)
    - [(ii) Which channels bring in such kind of traffic?](#typical_countries_channel)
    - [(b) (i) From which cities does the website experience typical session durations?](#typical_cities)
    - [(ii) Which channels bring in such kind of traffic?](#typical_cities_channel)
    - [(c) What kind of device are typically used to access the website?](#device)
    - [(d) What is the average session duration for every device type and from where do these users belong from?](#sess_device)
- [What kind of devices are being used to access the website?](#devices)
- [Where do the visitors come from?](#visitors)
- [Where are the most sessions generated?](#most_session)
- [How has the number of sessions for every channel, varied over time?](#session_channel)
- [What is the split between first time visitors and returning visitors?](#visitor_split)
- [Which cities having really low bouncerates and what kind of users are there?](#bounce)

## <a id = 'lib'><h4>Importing the essential libraries</h4></a>

In [ ]:
!pip install numpy 
!pip install pandas 
!pip install sqlite3
!pip install  matplotlib
!pip install  statistics
!pip install  plotly
!pip install  plotly

In [2]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
import statistics as stat
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)
import IPython

from IPython.display import HTML

import plotly.io as pio



path = 'C:\\Users\\soumy\\Desktop\\Project 1\\data.csv'

conn = sqlite3.connect('pb_data.sqlite')
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS pb_da_task''')
cur.execute(''' CREATE TABLE pb_da_task
                ('PK_Index' INTEGER PRIMARY KEY AUTOINCREMENT,
                 'yearMonth' CHAR(6),
                 'Country' VARCHAR(60),
                 'Region' VARCHAR(60),
                 'City' VARCHAR(60),
                 'channelGrouping' VARCHAR(20),
                 'deviceCategory' VARCHAR(10),
                 'userType' VARCHAR(20),
                 'users' INTEGER,
                 'sessions' INTEGER,
                 'avgSessionDuration' INTEGER,
                 'bounceRate' REAL)''')

read_data = pd.read_csv(path)
read_data.to_sql('pb_da_task', conn, if_exists='append', index = False) 

## <a id = 're'><h4>Reading and modifying the data</h4></a> 


In [24]:
table = pd.read_sql('''SELECT * FROM pb_da_task ''',conn)
table.insert(12, "totalSessionDuration", pd.read_sql('''SELECT avgSessionDuration * sessions FROM pb_da_task ''',conn),True)
table['Year']= table['yearMonth'].astype('str').apply(lambda x:x[:4]).astype('int')
table['Month']= table['yearMonth'].astype('str').apply(lambda x:x[4:]).astype('int')
table.head()

,PK_Index,yearMonth,Country,Region,City,channelGrouping,deviceCategory,userType,users,sessions,avgSessionDuration,bounceRate,totalSessionDuration,Year,Month
0,1,201801,(not set),(not set),(not set),Direct,desktop,New Visitor,8,8,21.25,75.0,170.0,2018,1
1,2,201801,(not set),(not set),(not set),Direct,desktop,Returning Visitor,3,5,0.00,100.0,0.0,2018,1
2,3,201801,(not set),(not set),(not set),Direct,mobile,New Visitor,1,1,261.00,0.0,261.0,2018,1
3,4,201801,(not set),(not set),(not set),Direct,mobile,Returning Visitor,1,1,0.00,100.0,0.0,2018,1
4,5,201801,(not set),(not set),(not set),Organic Search,desktop,New Visitor,8,8,357.75,50.0,2862.0,2018,1


<a href="#main">Go back to index</a>

## <a id = 'typical'><h4>How much time does someone typically spend on the web page?</h4></a>

In [4]:
table.totalSessionDuration.mean()/table.sessions.mean()

220.17379753147347

Typically people spend 3.7 minutes (approximately) on the webpage per session.  

Following the typical session duration, let us find out the top 10 countries and cities from where the typical usage has been observed.

<a href="#main">Go back to index</a>

### <a id = 'typical_countries'><h7>(a) (i) From which countries does the website experience typical session durations?</h7></a>

In [5]:
avg_country = pd.read_sql('''SELECT Country, deviceCategory, COUNT(avgSessionDuration) as avgSessionDuration FROM pb_da_task
                             GROUP BY country, deviceCategory, avgSessionDuration
                             HAVING avgSessionDuration BETWEEN 211 AND 220
                             AND Country <> '(not set)'
                             ORDER BY avgSessionDuration DESC
                             ''',conn)

avg10_country = avg_country[avg_country.Country.isin([x for x in avg_country.Country.unique()[:10]])]
ad = pd.crosstab(avg10_country.Country,avg10_country.deviceCategory, values=avg10_country.avgSessionDuration,aggfunc= sum, 
            margins = True,margins_name = 'Total').reset_index()
# ad

In [7]:
init_notebook_mode(connected=True)

avg_country.loc[avg_country.avgSessionDuration <= 2, 'Country'] = 'Other countries'
fig23 = px.pie(avg_country, values='avgSessionDuration', names='Country', 
             title='Typical session durations from across the globe.')

fig231 = go.Figure(go.Bar(
            x=ad.iloc[[1,2,3,8],4].tolist(),
            y=ad.iloc[[1,2,3,8],0].tolist(),
            orientation='h'))

fig231.update_layout(title='Top 4 countries',
                  yaxis_title='Countries',
                  xaxis_title='avgSessionDuration (seconds)',
                  height=600,
                  font=dict(size=16))

iplot(fig23, filename = 'basic-pie', config = config)
# plot(fig, filename = 'figure_1.html', config = config)
# display(HTML('figure_1.html'))

iplot(fig231, filename = 'basic-bar', config = config)

It turns out that users typically come from India, United States, France and Italy. These users tend to use the website, typically speaking, more than the other countries. Indian users typically spend 7.57 minutes while US users spend 5.78 minutes on an average on this page. 

Although the webpage is of Indian origin, the average session duration for US users are considerably higher than the typical session durations.

In order to get a better idea of how the users are coming in based on the typical session duration, lets look at what the channel groups have to say.

<a href="#main">Go back to index</a>

### <a id = 'typical_countries_channel'><h7>(ii) Which channels bring in such kind of traffic?</h7></a>

In [8]:
avg_country_c = pd.read_sql('''SELECT Country, channelGrouping, COUNT(avgSessionDuration) as avgSessionDuration FROM pb_da_task
                             GROUP BY country, channelGrouping, avgSessionDuration
                             HAVING avgSessionDuration BETWEEN 211 AND 220
                             AND Country <> '(not set)'
                             ORDER BY avgSessionDuration DESC
                             ''',conn)

avg10_country_c = avg_country_c[avg_country_c.Country.isin([x for x in avg_country_c.Country.unique()[:10]])]
ad_c = pd.crosstab(avg10_country_c.Country,avg10_country_c.channelGrouping, values=avg10_country_c.avgSessionDuration,
                 aggfunc= sum, margins = True,margins_name = 'Total').fillna(0).reset_index()
# ad_c

The following pie-chart shows the traffic for average session durations from different channel-groups and their proportions, while the following bar graph depicts the break-up based on the channel-groups.

In [10]:
init_notebook_mode(connected=True)
avg_country_c.loc[avg_country_c.avgSessionDuration <= 2, 'Country'] = 'Other countries'

fig50 = px.pie(avg_country_c, values='avgSessionDuration', names='channelGrouping', 
             title='Typical session durations from channel groups.')

fig60 = go.Figure()

name=ad_c.iloc[[1,3,8,9],0].tolist()

fig60 = go.Figure(data=[
    go.Bar(name='Other', x=name, y=ad_c.iloc[[1,3,8,9],1].tolist()),
    go.Bar(name='Direct', x=name, y=ad_c.iloc[[1,3,8,9],2].tolist()),
    go.Bar(name='Organic Search', x=name, y=ad_c.iloc[[1,3,8,9],3].tolist()),
    go.Bar(name='Paid Search', x=name, y=ad_c.iloc[[1,3,8,9],4].tolist()),
    go.Bar(name='Referral', x=name, y=ad_c.iloc[[1,3,8,9],5].tolist()),
    go.Bar(name='Social', x=name, y=ad_c.iloc[[1,3,8,9],6].tolist())
])

fig60.update_layout(title='Channel group comparison for the top 4 countries based on the typical session duration',
                  xaxis_title='Countries',
                  yaxis_title='avgSessionDuration (seconds)',
                  height=600,
                  font=dict(size=14))

iplot(fig50, filename = 'basic-pie', config = config)
fig60.update_layout(barmode='stack')#, xaxis_tickangle=-45)
iplot(fig60, filename = 'basic-bar', config = config)

In the pie chart we can see that organic search and direct seems to be quite prevalent. This means that users visitng the website are either already aware of the website or one of the recommended search results led them to the website.

When we look at the top 4 countries from where the users are visiting, we see paid search seems to be quite popular in India, while referral in United States, other than organic search and direct channels.

Since India is the country of origin, the traffic via paid search channel is expected. These users are most probably new users who landed on the page via an ad. In United States, it is safe to assume that quite a big share of users from the referral channel are new visitors who have been referred by someone. The visitor traffic from direct and organic search channels are probably due to returning visitors. Traffic from display and email channels seem to completely absent.

<a href="#main">Go back to index</a>

### <a id = 'typical_cities'><h7>(b) (i) From which cities does the website experience typical session durations?</h7></a>

In [13]:
avg_city = pd.read_sql('''SELECT City, deviceCategory, COUNT(avgSessionDuration) as avgSessionDuration FROM pb_da_task
                             GROUP BY City, deviceCategory, avgSessionDuration
                             HAVING avgSessionDuration BETWEEN 211 AND 220
                             AND City <> '(not set)'
                             ORDER BY avgSessionDuration DESC
                             ''',conn)
avg10_city = avg_city[avg_city.City.isin([x for x in avg_city.City.unique()[:10]])]
ac = pd.crosstab(avg10_city.City,avg10_city.deviceCategory, values=avg10_city.avgSessionDuration,aggfunc= sum, 
            margins = True,margins_name = 'Total').fillna(0).reset_index()
# ac

In [15]:
init_notebook_mode(connected=True)

avg_city.loc[avg_city.avgSessionDuration < 2, 'City'] = 'Other cities'
fig_1 = go.Figure(data=[go.Pie(values=avg_city.avgSessionDuration, labels=avg_city.City, rotation = 90)])

fig_1.update_layout(title_text="Typical session durations across cities worldwide.",titlefont_size=17)

fig_12 = go.Figure(go.Bar(
            x=ac.iloc[[3,5,7,8],4].tolist(),
            y=ac.iloc[[3,5,7,8],0].tolist(),
            orientation='h'))

fig_12.update_layout(title='Top 4 cities',
                  yaxis_title='Cities',
                  xaxis_title='avgSessionDuration (seconds)',
                  height=600,
                  font=dict(size=16))

iplot(fig_1, filename = 'basic-pie', config = config)
iplot(fig_12, filename = 'basic-bar', config = config)

The typical usage when filtered by cities happen to be quite low, where the highest session duration is only 7 seconds as seen above.

<a href="#main">Go back to index</a>

### <a id = 'typical_cities_channel'><h7>(ii) Which channels bring in such kind of traffic?</h7></a>

In [17]:
avg_city_c = pd.read_sql('''SELECT City, channelGrouping, COUNT(avgSessionDuration) as avgSessionDuration FROM pb_da_task
                             GROUP BY City, channelGrouping, avgSessionDuration
                             HAVING avgSessionDuration BETWEEN 211 AND 220
                             AND City <> '(not set)'
                             ORDER BY avgSessionDuration DESC
                             ''',conn)

avg10_city_c = avg_city_c[avg_city_c.City.isin([x for x in avg_city_c.City.unique()[:10]])]
a_c = pd.crosstab(avg10_city_c.City,avg10_city_c.channelGrouping, values=avg10_city_c.avgSessionDuration,
                 aggfunc= sum, margins = True,margins_name = 'Total').fillna(0).reset_index()
# a_c

In [18]:
init_notebook_mode(connected=True)

avg_city_c.loc[avg_city_c.avgSessionDuration <= 2, 'City'] = 'Other cities'

fig5 = px.pie(avg_city_c, values='avgSessionDuration', names='channelGrouping', 
             title='Typical session durations from channel groups, for all cities.')

fig6 = go.Figure()

name = a_c.iloc[[3,5,6,7],0].tolist()

fig6 = go.Figure(data=[
    go.Bar(name='Direct', x=name, y=a_c.iloc[[3,5,6,7],1].tolist()),
    go.Bar(name='Organic Search', x=name, y=a_c.iloc[[3,5,6,7],2].tolist()),
    go.Bar(name='Paid Search', x=name, y=a_c.iloc[[3,5,6,7],3].tolist()),
    go.Bar(name='Referral', x=name, y=a_c.iloc[[3,5,6,7],4].tolist()),
    go.Bar(name='Social', x=name, y=a_c.iloc[[3,5,6,7],5].tolist())
])

fig6.update_layout(title='Channel group comparison for the top 4 cities based on the typical session duration',
                  xaxis_title='Cities',
                  yaxis_title='avgSessionDuration (seconds)',
                  height=600,
                  font=dict(size=14))

iplot(fig5, filename = 'basic-bar', config = config)
fig6.update_layout(barmode='stack')#, xaxis_tickangle=-45)
iplot(fig6, filename = 'basic-bar', config = config)

The pie chart above shows the channel-group proportions for the typical session durations. The proportion of organic search and direct channels seem to be the highest. 

In case of the traffic experienced from the cities, especially the top 4, there is not much of a difference.

In cities like Jaipur and Ahmedabad there is a higher probability of returning visitor because the visitors are generally coming from direct channel, more than the other channels. In cities outside India, like Accra, the organic search channels seems to be typical. This may mean that new visitors might me landing on the website more than returning visitors.

<a href="#main">Go back to index</a>

### <a id = 'device'><h7>(c) What kind of device are typically used to access the website?</h7> </a>

In [19]:
avg_UT = pd.read_sql('''SELECT Country, userType, deviceCategory,COUNT(avgSessionDuration) as avgSessionDuration 
                        FROM pb_da_task
                        GROUP BY country, userType, deviceCategory,avgSessionDuration
                        HAVING avgSessionDuration BETWEEN 211 AND 220
                        ORDER BY avgSessionDuration DESC''',conn)

# pd.crosstab(avg_UT.userType,avg_UT.deviceCategory, values=avg_UT.avgSessionDuration,aggfunc= sum,
#             margins = True )

In [20]:
init_notebook_mode(connected=True)

fig = px.bar(avg_UT, x="deviceCategory", y="avgSessionDuration",color = "userType",barmode='group',height=500)
fig.update_layout(title='Device usage based on typical session duration',
                  xaxis_title='Device type',
                  yaxis_title='avgSessionDuration (seconds)',
                  height=450,
                  font=dict(size=14))

iplot(fig, filename = 'basic-bar', config = config)

The above graph depicts the kind of devices are used by both new and returning visitors. New visitors, typically, access the webpage via desktop but returning visitors have an affinity towards the mobile device too. Returning visitors prefer mobile phones almost as much as they prefer desktops.

<a href="#main">Go back to index</a>

### <a id = 'sess_device'><h7> (d) What is the average session duration for every device type and from where do these users belong from?</h7></a>

In [21]:
aggregate_table = table.groupby(['deviceCategory']).agg(
                                    max_duration = pd.NamedAgg(column='avgSessionDuration', aggfunc = max),
                                    min_duration = pd.NamedAgg(column='avgSessionDuration', aggfunc = min),
                                    total_duration = pd.NamedAgg(column='avgSessionDuration', aggfunc = sum),
                                    average_duration = pd.NamedAgg(column='avgSessionDuration', aggfunc = stat.mean)
                                    ).reset_index()
aggregate_table

,deviceCategory,max_duration,min_duration,total_duration,average_duration
0,desktop,23220.0,0.0,3.337276e+07,282.578169
1,mobile,8145.0,0.0,1.124836e+07,122.747749
2,tablet,8541.0,0.0,5.108323e+06,229.577245


The average session duration for desktops is 4.71 minutes, mobiles is 2.05 minutes and for tablets is 3.83 minutes, approximately. 

Although people do not use tablets as much as they use desktops, they do spend a lot of time consuming content. Mobile users may seem high in number but they spend comparatively less time on their devices.

#### Locations -

In [16]:
a=table[table.avgSessionDuration.between(282.4,282.6)]
a[a.deviceCategory =='desktop'].max().reset_index().T.drop([0,1,3,4,11,12,13,14],axis=1)

,2,5,6,7,8,9,10
index,Country,channelGrouping,deviceCategory,userType,users,sessions,avgSessionDuration
0,Vietnam,Referral,desktop,Returning Visitor,19,19,282.6


Desktop users typically belong from Vietnam.

In [17]:
b = table[table.avgSessionDuration.between(122.6,122.8)]
b[b.deviceCategory =='mobile'].max().reset_index().T.drop([0,1,3,4,11,12,13,14],axis=1)

,2,5,6,7,8,9,10
index,Country,channelGrouping,deviceCategory,userType,users,sessions,avgSessionDuration
0,United States,Social,mobile,Returning Visitor,91,92,122.75


Mobile users typically belong from US.

In [18]:
c = table[table.avgSessionDuration.between(122.6,122.8)]
c[c.deviceCategory =='tablet'].max().reset_index().T.drop([0,1,3,4,11,12,14],axis=1)

,2,5,6,7,8,9,10,13
index,Country,channelGrouping,deviceCategory,userType,users,sessions,avgSessionDuration,Year
0,India,Organic Search,tablet,Returning Visitor,3,4,122.75,2018


Tablet users, also, typically belong from India.

Next, let us assess the effect of devices on all users, including the typical users.

<a href="#main">Go back to index</a>

## <a id='devices'> <h4>What kind of devices are being used to access the website?</h4></a>

The visualisations below have grouped in two ways. The first two graphs show the kind of usage for every channel-group traffic.
The first graph shows the traffic that new visitors bring while the second one shows the traffic due to returning visitors. The final one compares the traffic based on the type of device both the kind of visitors prefer. The comparison has been done with the number of sessions.

In [99]:
f = pd.pivot_table(data=table, index='channelGrouping', columns=['userType','deviceCategory'],values='sessions', 
               aggfunc =sum,margins=True).fillna(0).reset_index()

# f

In [100]:
init_notebook_mode(connected=True)

devices = ['desktop','mobile','tablet']
visitor = ['New Visitor','Returning Visitor']

fig1 = go.Figure(data=[
    go.Bar(name=devices[0], x=f.channelGrouping[:8], y=f[visitor[0]][devices[0]][:8].tolist()),
    go.Bar(name=devices[1], x=f.channelGrouping[:8], y=f[visitor[0]][devices[1]][:8].tolist()),
    go.Bar(name=devices[2], x=f.channelGrouping[:8], y=f[visitor[0]][devices[2]][:8].tolist()),
])

fig1.update_layout(title='New visitors from all channel-groups',
                  yaxis_title='Number of sessions',
                  xaxis_title='Channels',
                  height=600,
                  font=dict(size=18))

fig2 = go.Figure(data=[
    go.Bar(name=devices[0], x=f.channelGrouping[:8], y=f[visitor[1]][devices[0]][:8].tolist()),
    go.Bar(name=devices[1], x=f.channelGrouping[:8], y=f[visitor[1]][devices[1]][:8].tolist()),
    go.Bar(name=devices[2], x=f.channelGrouping[:8], y=f[visitor[1]][devices[2]][:8].tolist()),
])

fig2.update_layout(title='Returning visitors from all channel-groups',
                  yaxis_title='Number of sessions',
                  xaxis_title='Channels',
                  height=600,
                  font=dict(size=18))

fig3 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig3.add_trace(go.Pie(labels=devices, values=f.iloc[8,1:4].tolist(), name=visitor[0]),1, 1)
fig3.add_trace(go.Pie(labels=devices, values=f.iloc[8,4:7].tolist(), name=visitor[1]),1, 2) 

fig3.update_traces(hole=.5, hoverinfo="label+percent+name")

fig3.update_layout(
    title_text= "Device usage based on user type.",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='New', x=0.198, y=0.5, font_size=20, showarrow=False),
                 dict(text='Returning', x=0.84, y=0.5, font_size=20, showarrow=False)])

fig1.update_layout(barmode='stack')
fig2.update_layout(barmode='stack')

iplot(fig1, filename = 'basic-bar', config = config)
iplot(fig2, filename = 'basic-bar', config = config)
iplot(fig3, filename = 'basic-pie', config = config)

Visitors coming in from all channels prefer mobile more than all the other devices, yet returning users have an affinity towards desktops. That may be so because they are probably spending more time to view the content than new visitors. 

Organic search, paid search and direct channels seem to bring most of traffic for the website regardless of the kind of visitor. New visitors have more number of mobile sessions in organic than returning visitors do. Although the number of desktop sessions for returning visitors is more than mobile sessions, a significant number of returning visitors still prefer the mobile. This may be probably be because these users may only have access to a mobile and maybe they cannot afford another device. 

<a href="#main">Go back to index</a>

## <a id = 'visitors'><h4>Where do the visitors come from?</h4></a>

The goal here is to understand the usage based on the number of users from all the countries and cities. The first two pie charts show the distribution across countries and cities, respectively. The comparison has been done based on the number of users.

In [101]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
df1 = pd.read_sql('''SELECT Country, SUM(users) AS users
                               FROM pb_da_task
                               WHERE country <> '(not set)'
                               GROUP BY country
                               ORDER BY users DESC
                               ''',conn)

df1.loc[df1.users <= 10000, 'Country'] = 'Other countries'
fig = px.pie(df1, values='users', names='Country', title='Users from across the globe.')

df2 = pd.read_sql('''SELECT City, SUM(users) AS users
                               FROM pb_da_task
                               WHERE City <> '(not set)'
                               GROUP BY City
                               ORDER BY users DESC
                               ''',conn)

df2.loc[df2.users <= 40000, 'City'] = 'Other cities'
fig1 = px.pie(df2, values='users', names='City', title='Users from across all cities.')

# fig.show()
iplot(fig, filename = 'basic-pie', config = config)
iplot(fig1, filename = 'basic-pie', config = config)
# fig1.show()

84.2% of all users come from India. So the country from where most users come is India wherein Bangalore is the city where most users visit the webpage. Followed by India, USA and Phillipines are the countries from where most visitors come from.

<a href="#main">Go back to index</a>

## <a id = 'most_session'><h4>Where are the most sessions generated?</h4></a>

The two bar charts below show only the top 10 countries and cities out of all the locations from where most sessions were generated. The first bar chart shows the top 10 countries while the chart below shows the top cities, based the number maximum sessions.

In [102]:
{
    "tags": [
        "hide_input",
    ]
}
max_country = pd.read_sql('''SELECT Country, channelGrouping,  COUNT(sessions) as sessions FROM pb_da_task
                             GROUP BY Country,channelGrouping, sessions
                             HAVING Country <> '(not set)'
                             AND channelGrouping <> '(Other)'
                             ORDER BY sessions DESC
                             ''',conn)

max_city = pd.read_sql('''SELECT City, channelGrouping,  COUNT(sessions) as sessions FROM pb_da_task
                             GROUP BY City,channelGrouping, sessions
                             HAVING City <> '(not set)'
                             AND channelGrouping <> '(Other)'
                             ORDER BY sessions DESC
                             ''',conn)

top10_country = max_country[max_country.Country.isin([x for x in max_country.Country.unique()[:10]])]
top10_city = max_city[max_city.City.isin([x for x in max_city.City.unique()[:10]])]

a = pd.crosstab(top10_country.Country,top10_country.channelGrouping,values=top10_country.sessions,
            aggfunc= sum,margins = True, margins_name= 'Total').fillna(0).reset_index()

b = pd.crosstab(top10_city.City,top10_city.channelGrouping,values=top10_city.sessions,
            aggfunc= sum,margins = True, margins_name= 'Total').fillna(0).reset_index()
# b

In [103]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
fig1 = px.bar(x=a.Country[:9], y=a.Total[:9],
             color=a.Total[:9], height=600)

fig1.update_layout(title='Top 10 countries.',
                  yaxis_title='Number of sessions',
                  xaxis_title='Countries',
                  font=dict(size=18))

fig2 = px.bar(x=b.City[:9], y=b.Total[:9],
             color=b.Total[:9], height=600)

fig2.update_layout(title='Top 10 cities.',
                  yaxis_title='Number of sessions',
                  xaxis_title='Cities',
                  font=dict(size=18))


iplot(fig1, filename = 'basic-bar', config = config)

iplot(fig2, filename = 'basic-bar', config = config)

India, UK, Canada and Italy are the countries from where the most sessions were generated, while Dublin(Canada), Cambridge, Columbus and Athens (US) are the cities with most sessions.

<a href="#main">Go back to index</a>

## <a id = 'session_channel'><h4>How has the number of sessions for every channel, varied over time?</h4></a>

In this section I have tried to find out which month had the most sessions and from which channel groups have these sessions come from. The first figure shows the analysis for 2018 and the second one shows the analysis for 2019. The third figure shows how the number of sessions for every channel group varied from 2018 to 2019.

In [104]:
{
    "tags": [
        "hide_input",
    ]
}
channel_year18 = pd.read_sql('''SELECT yearMonth, channelGrouping, Country, COUNT(sessions) AS sessions
                                  FROM pb_da_task
                                  GROUP BY yearMonth, channelGrouping,Country, sessions
                                  HAVING SUBSTR(yearMonth, 1, 4)='2018'
                                  AND channelGrouping <> '(Other)'
                                  ORDER BY yearMonth
                               ''',conn)

channel_year19 = pd.read_sql('''SELECT yearMonth, channelGrouping, Country, COUNT(sessions) AS sessions
                                  FROM pb_da_task
                                  GROUP BY yearMonth, channelGrouping,Country, sessions
                                  HAVING SUBSTR(yearMonth, 1, 4)='2019'
                                  AND channelGrouping <> '(Other)'
                                  ORDER BY yearMonth
                               ''',conn)

channels_2018 = pd.pivot_table(data=channel_year18, index='channelGrouping', columns='yearMonth',values='sessions', 
               aggfunc =sum,margins=True, margins_name = 'Total').fillna(0).reset_index()

channels_2019 = pd.pivot_table(data=channel_year19, index='channelGrouping', columns='yearMonth',values='sessions', 
               aggfunc =sum,margins=True, margins_name = 'Total').fillna(0).reset_index()

In [105]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 
        'August', 'September', 'October', 'November', 'December']

fig1 = go.Figure(data=[
    go.Bar(name='Direct', x=months, y=channels_2019.iloc[0,1:13].tolist()),
    go.Bar(name='Display', x=months, y=channels_2019.iloc[1,1:13].tolist()),
    go.Bar(name='Email', x=months, y=channels_2019.iloc[2,1:13].tolist()),
    go.Bar(name='Organic Search', x=months, y=channels_2019.iloc[3,1:13].tolist()),
    go.Bar(name='Paid Search', x=months, y=channels_2019.iloc[4,1:13].tolist()),
    go.Bar(name='Referral', x=months, y=channels_2019.iloc[5,1:13].tolist()),
    go.Bar(name='Social', x=months, y=channels_2019.iloc[6,1:13].tolist())
])

fig2 = go.Figure(data=[
    go.Bar(name='Direct', x=months, y=channels_2018.iloc[0,1:13].tolist()),
    go.Bar(name='Email', x=months, y=channels_2018.iloc[1,1:13].tolist()),
    go.Bar(name='Organic Search', x=months, y=channels_2018.iloc[2,1:13].tolist()),
    go.Bar(name='Paid Search', x=months, y=channels_2018.iloc[3,1:13].tolist()),
    go.Bar(name='Referral', x=months, y=channels_2018.iloc[4,1:13].tolist()),
    go.Bar(name='Social', x=months, y=channels_2018.iloc[5,1:13].tolist())
])

#Update the labels
fig1.update_layout(title='Traffic from all channel groups - 2018',
                  yaxis_title='Number of sessions',
                  xaxis_title='Months',
                  font=dict(size=18))
fig2.update_layout(title='Traffic from all channel groups - 2019',
                  yaxis_title='Number of sessions',
                  xaxis_title='Months',
                  font=dict(size=18))

# Change the bar mode
fig1.update_layout(barmode='stack')
fig2.update_layout(barmode='stack')

# fig1.show()
iplot(fig1, filename = 'stack-bar', config = config)
iplot(fig2, filename = 'stack-bar', config = config)
# fig2.show()

In 2018, April experienced the highest number of sessions. The traffic originated mostly from organic search and referral. While in 2019, March experienced the most traffic. This time the traffic came, mostly, from paid search and organic search channels.

Based on the above facts the following inferences maybe made-

 - Referrals and search engine optimisation has played one of the key roles in boosting website traffic. 
 
 
 - The highest number of sessions are generally observed during the first half of the year, especially between February and      June. This kind of behavior maybe because most of the users might be kids who are under 18 and are attending school. During this period the exams happen and get over, new sessions begin and eventually there are summer vacations. 

In [106]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=channels_2018.channelGrouping[:6],
    y=channels_2018.Total[:6],
    name='2018',
    marker_color='indianred'
))
fig3.add_trace(go.Bar(
    x=channels_2019.channelGrouping[:7],
    y=channels_2019.Total[:7],
    name='2019',
    marker_color='lightsalmon'
))

fig3.update_layout(title='Channel group traffic from 2018 to 2019',
                  yaxis_title='Number of sessions',
                  xaxis_title='Channel groups',
                  font=dict(size=18))

fig3.update_layout(barmode='group', xaxis_tickangle=-45)

# fig3.show()
iplot(fig3, filename = 'group-bar', config = config)

The number of sessions from direct and refferal channels have significantly increased. Whereas there has been a significant decrease in the number of sessions from paid search channels.

An increase in direct channel traffic may suggest a spike in the number of returning visitors because they may already be aware of the website. While increase in the referral link may mean more new visitors given the traffic occurs only after being reffered to.

The decrease in number of sessions from paid search suggests that users are less likely to click on a link which serves as advertisements for this website.

<a href="#main">Go back to index</a>

## <a id = 'visitor_split'><h4>What is the split between first time visitors and returning visitors?</h4></a>

I am trying to determine how has number of sessions, the average session duration and the bouncerate varied among new visitors and returning visitors from 2018 to 2019. For every chart below a comparison for new vs returning visitor for both 2018 and 2019 has been done to compare sessions, the average session duration and the bouncerate,respectively.

In [107]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
user_val = table[['sessions','userType','avgSessionDuration','bounceRate','Year']]

a = pd.crosstab(user_val.userType,user_val.Year, values = user_val.sessions,
            aggfunc= sum,normalize ='columns',margins = True, margins_name= 'Total').fillna(0).reset_index()

fig3 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig3.add_trace(go.Pie(labels=a.userType.tolist(), values=a.iloc[:,1].tolist(), name=a.columns.tolist()[1]),1, 1)
fig3.add_trace(go.Pie(labels=a.userType.tolist(), values=a.iloc[:,2].tolist(), name=a.columns.tolist()[2]),1, 2) 

fig3.update_traces(hole=.6, hoverinfo="label+percent+name")

fig3.update_layout(
    title_text= "Variation of sessions from 2018 to 2019",
    annotations=[dict(text='2018', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='2019', x=0.81, y=0.5, font_size=20, showarrow=False)])

iplot(fig3, filename = 'donut-pie', config = config)

32.46% of all users, in 2018, have returned to the webpage while in 2019 34.60% have. So there has been a 2.14% increase in the rate of returning users.

In [108]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
b = pd.crosstab(user_val.userType,user_val.Year, values = user_val.avgSessionDuration,
            aggfunc= sum,normalize = 'columns',margins = True, margins_name= 'Total').fillna(0).reset_index()

fig3 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig3.add_trace(go.Pie(labels=b.userType.tolist(), values=b.iloc[:,1].tolist(), name=b.columns.tolist()[1]),1, 1)
fig3.add_trace(go.Pie(labels=b.userType.tolist(), values=b.iloc[:,2].tolist(), name=b.columns.tolist()[2]),1, 2) 

fig3.update_traces(hole=.6, hoverinfo="label+percent+name")

fig3.update_layout(
    title_text= "Variation of avg session durations from 2018 to 2019",
    annotations=[dict(text='2018', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='2019', x=0.81, y=0.5, font_size=20, showarrow=False)])

iplot(fig3, filename = 'donut-pie', config = config)

The duration for which users have been accessing the webpage over the 2 years have not varied drastically. They have more or less remained the same. 

In [109]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
c = pd.crosstab(user_val.userType,user_val.Year, values = user_val.bounceRate,
            aggfunc= sum,normalize ='columns',margins = True, margins_name= 'Total').fillna(0).reset_index()

fig3 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig3.add_trace(go.Pie(labels=c.userType.tolist(), values=c.iloc[:,1].tolist(), name=c.columns.tolist()[1]),1, 1)
fig3.add_trace(go.Pie(labels=c.userType.tolist(), values=c.iloc[:,2].tolist(), name=c.columns.tolist()[2]),1, 2) 

fig3.update_traces(hole=.6, hoverinfo="label+percent+name")

fig3.update_layout(
    title_text= "Variation of bounce-rate from 2018 to 2019",
    annotations=[dict(text='2018', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='2019', x=0.81, y=0.5, font_size=20, showarrow=False)])

iplot(fig3, filename = 'donut-pie', config = config)

The average bouncerate for returning users in 2018 was 28.45% which has increased to 56.04% in 2019. The bounce rate has increased by 27.6%, i.e. the bounce rate for returning users have doubled in a year. The returning users do not have enough intensive to stay on the web page for long.

While average bouncerate for new users in 2018 was 71.55% which has decreased to 43.96% in 2019. Hence there has been a significant decrease in bouncerate for the new visitors.

The variations above suggest that measures have been taken to make the content more engaging for the new users without significantly harming the returning visitor traffic over the considered period of time. 

<a href="#main">Go back to index</a>

## <a id = 'bounce'><h4> Which cities having really low bouncerates and what kind of users are there?</h4></a> 

The following analysis was done find out some of the cities with really low bounce rates. Having really low bounce rate would mean that people from these places are really interested in accesing the webpage.

In [110]:
{
    "tags": [
        "hide_input",
    ]
}
ag = table.loc[table.bounceRate.between(0,35)].groupby('bounceRate').min().reset_index()
ag=ag[ag.City != '(not set)']
city_min = ag[ag.City.isin([x for x in ag.City.unique()[:10]])]
ft = pd.pivot_table(data=city_min, index='City', columns=['userType','deviceCategory'],values='bounceRate', 
               aggfunc =lambda x:x.sum()/(city_min.bounceRate.sum()),margins=True,margins_name='Total').fillna(0).reset_index()

In [34]:
{
    "tags": [
        "hide_input",
    ]
}
# vals1 = [sum([ft.iloc[10,1:2].tolist()[0],ft.iloc[10,4:5].tolist()[0]]),
# sum([ft.iloc[10,2:3].tolist()[0],ft.iloc[10,5:6].tolist()[0]]),
# sum([ft.iloc[10,3:4].tolist()[0],ft.iloc[10,6:7].tolist()[0]])]
# fig2_3 = px.pie(values=vals1, names=ft['New Visitor'].columns.tolist(),
#                 title= "Proportion of bounce-rates for every type of device.")
# fig2_3.show()

In [111]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
fig = px.pie(values=ft.Total[:10], names=ft.City[:10], title='Top 10 cities with minimum bouncerates')
iplot(fig, filename = 'basic-pie', config = config)

Chennai has the lowest bouncerates, followed Noida and Surat. Visitors from these places tend to stay back more.

In [112]:
{
    "tags": [
        "hide_input",
    ]
}
init_notebook_mode(connected=True)
vals=[sum(ft.iloc[10,1:4].tolist()),sum(ft.iloc[10,4:7].tolist())]
fig = px.pie(values=vals, names=['New Visitor', 'Returning Visitor'], 
             title='Split between type of user for minimum bouncerates')
iplot(fig, filename = 'basic-pie', config = config)

The proportion of new and returning visitors are mostly simillar for low bouncerates. This means that a significant number, if not all, of the new visitors are returning as returning visitors for these bouncerates.

These locations and the ones nearby may have more potential visitors in the near future. 

<a href="#main">Go back to index</a>

### Thank you for your time!